In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# To prevent the error in the training section
tf.config.run_functions_eagerly(True)

**Exploring Data**

In [4]:
# Reading the dataset
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='ISO-8859-1')

df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [5]:
# Check for null values
df.isna().sum()

v1               0
v2               0
Unnamed: 2    5522
Unnamed: 3    5560
Unnamed: 4    5566
dtype: int64

In [6]:
# Dropping unnamed columns
df = df[['v1', 'v2']]

df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [7]:
# Check for empty space strings
blanks = []

for i, v1, v2 in df.itertuples():
    if type(v2) == str and v2.isspace():
        blanks.append(i)
        
if len(blanks) > 0:
    print(f'There are len(blanks) empty space strings in the dataset')
    df.drop(blanks, inplace=True)

In [8]:
# Checking for balancement
df['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
# Get the max length of sequences
max_seq_length = df['v2'].str.len().max()

print("Max sequence length:", max_seq_length)

Max sequence length: 910


In [10]:
# Get the number of unique words in the whole dictionary
words = set(word.lower() for sentence in df['v2'] for word in sentence.split())
n_unique_words = len(words)

print("Unique words:", n_unique_words)

Unique words: 13496


In [11]:
# Separating X and y
X = df['v2']
y = df['v1']

display(X, y)

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                Will Ì_ b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: v2, Length: 5572, dtype: object

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: v1, Length: 5572, dtype: object

In [12]:
# Encoding the Labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y = le.fit_transform(y)

display(y)

array([0, 0, 1, ..., 0, 0, 0])

In [13]:
# Preparing the data
max_len = 150
num_words = 1000

In [14]:
# Tokenizing
tok = Tokenizer(num_words=num_words)
tok.fit_on_texts(df['v2'])


In [15]:
# Train and Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [16]:
X_train_mat = tok.texts_to_sequences(X_train)
X_test_mat = tok.texts_to_sequences(X_test)
# Pad sequences to the same length
X_train_padded = pad_sequences(X_train_mat, maxlen=max_len)
X_test_padded = pad_sequences(X_test_mat, maxlen=max_len)
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

**Model**

In [17]:
# Create NN Architectures

# RNN
def create_rnn_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(SimpleRNN(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# LSTM
def create_lstm_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(LSTM(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model

# GRU
def create_gru_model(units, dropout_rate):
    model = Sequential()
    model.add(Embedding(num_words, 100, input_length=max_len))
    model.add(GRU(units))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [18]:
# Define hyperparameters
units = 64
dropout_rate = 0.5

**Compiling**

In [19]:
# Create and compile the models

# RNN
rnn_model = create_rnn_model(units, dropout_rate)
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# LSTM
lstm_model = create_lstm_model(units, dropout_rate)
lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# GRU
gru_model = create_gru_model(units, dropout_rate)
gru_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [20]:
# Train the models with early stopping
early_stopping = EarlyStopping(patience=3, restore_best_weights=True)

**Training**

In [21]:
# Train the models

# RNN
rnn_history = rnn_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, 
                            validation_split=0.2, callbacks=[early_stopping])

# LSTM
lstm_history = lstm_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, 
                              validation_split=0.2, callbacks=[early_stopping])

# GRU
gru_history = gru_model.fit(X_train_padded, y_train, epochs=10, batch_size=128, 
                            validation_split=0.2, callbacks=[early_stopping])


Epoch 1/10


/opt/conda/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:254: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


25/25 [==============================] - 13s 417ms/step - loss: 0.3421 - accuracy: 0.8840 - val_loss: 0.1851 - val_accuracy: 0.9333
Epoch 2/10
25/25 [==============================] - 10s 415ms/step - loss: 0.1114 - accuracy: 0.9699 - val_loss: 0.1095 - val_accuracy: 0.9705
Epoch 3/10
25/25 [==============================] - 10s 416ms/step - loss: 0.0799 - accuracy: 0.9785 - val_loss: 0.1159 - val_accuracy: 0.9603
Epoch 4/10
25/25 [==============================] - 10s 421ms/step - loss: 0.0507 - accuracy: 0.9862 - val_loss: 0.0692 - val_accuracy: 0.9808
Epoch 5/10
25/25 [==============================] - 10s 412ms/step - loss: 0.0352 - accuracy: 0.9910 - val_loss: 0.0738 - val_accuracy: 0.9808
Epoch 6/10
25/25 [==============================] - 10s 419ms/step - loss: 0.0242 - accuracy: 0.9955 - val_loss: 0.0818 - val_accuracy: 0.9782
Epoch 7/10
25/25 [==============================] - 10s 416ms/step - loss: 0.0340 - accuracy: 0.9894 - val_loss: 0.0714 - val_accuracy: 0.9808
Epoch 1/10

**Metrics**

In [22]:
rnn_val_acc = max(rnn_history.history['val_accuracy'])
lstm_val_acc = max(lstm_history.history['val_accuracy'])
gru_val_acc = max(gru_history.history['val_accuracy'])
max_val_acc = max(rnn_val_acc, lstm_val_acc, gru_val_acc)


print("RNN Validation Accuracy:", rnn_val_acc)
print("LSTM Validation Accuracy:", lstm_val_acc)
print("GRU Validation Accuracy:", gru_val_acc)
print('-' * 50)
print("Max Validation Accuracy:", max_val_acc)

RNN Validation Accuracy: 0.9807692170143127
LSTM Validation Accuracy: 0.985897421836853
GRU Validation Accuracy: 0.9846153855323792
--------------------------------------------------
Max Validation Accuracy: 0.985897421836853
